In [11]:
using JuMP
using Gurobi
using LinearAlgebra

In [14]:
model = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(model, x[1:4]>=0)

# objective
profit = 50*x[1]+40*x[2]+60*x[3]+30*x[4]

# Time constraint
a = [120, 100, 180, 140]
b = 5000
@constraint(model, dot(a, x)<=b)
@constraint(model, 2*x[1]+3*x[2]+6*x[3]+1*x[4] <= 80)
@constraint(model, 3*x[1]+2*x[2]+2*x[3]+0*x[4] <= 40)
@objective(model, Max, profit)
optimize!(model)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [16]:
model2 = Model(with_optimizer(Gurobi.Optimizer, OutputFlag=0))
@variable(model2, x[1:4]>=0)
E = diagm(0=>a)*0.3
@constraint(model2, dot(a, x)+norm(sqrt(E)*x)<=b)
@constraint(model2, 2*x[1]+3*x[2]+6*x[3]+1*x[4] <= 80)
@constraint(model2, 3*x[1]+2*x[2]+2*x[3]+0*x[4] <= 40)
@objective(model2, Max, profit)
optimize!(model2)

Academic license - for non-commercial use only


ErrorException: JuMP no longer performs automatic transformation of `norm()` expressions into second-order cone constraints. They should now be expressed using the SecondOrderCone() set. For example, `@constraint(model, norm(x) <= t)` should now be written as `@constraint(model, [t; x] in SecondOrderCone())`

In [ ]:
function wc_val(a, E, x)
    return a+E*x/sqrt(dot(x, E*x))
end

relaxed_constrs = nominal_constrs
for k in 1:20
    xk=x.value
    a_wc = wc_val(a, E, xk)
    optimize!()

In [ ]:
########################
## ROBUST FORMULATION ##
########################

# Time constraint ellipse
E = diagm(0=>a)*0.3

robust_constrs = [
    x>=0,
    dot(a, x)+norm(sqrt(E)*x)<=b,
    2*x[1]+3*x[2]+6*x[3]+1*x[4] <= 80,
    3*x[1]+2*x[2]+2*x[3]+0*x[4] <= 40,
]

robust_problem = maximize(profit, robust_constrs)
solve!(robust_problem, ECOSSolver(verbose=false))

println("Optimal Robust Solution: ", x.value)

#############################
## LARGE SCALE FORMULATION ##
#############################

function wc_val(a, E, x)
    return a+E*x/sqrt(dot(x, E*x))
end

relaxed_constrs = nominal_constrs
for k in 1:20

    xk=x.value
    a_wc = wc_val(a, E, xk)
    global relaxed_constrs = relaxed_constrs + [dot(a_wc,x)<=b]
    relaxed_problem = maximize(profit, relaxed_constrs)
    solve!(relaxed_problem, ClpSolver())

    println("Relaxed Solution (iteration " * string(k) *") : "* string(x.value)) 
end
